## Pengantar ke Regresi - Pelajaran 1

#### Memahami dalam konteks

✅ Ada banyak jenis metode regresi, dan pilihan metode tergantung pada jawaban yang ingin Anda cari. Jika Anda ingin memprediksi tinggi badan yang mungkin untuk seseorang berdasarkan usia tertentu, Anda akan menggunakan `linear regression`, karena Anda mencari **nilai numerik**. Jika Anda ingin mengetahui apakah jenis masakan tertentu harus dianggap vegan atau tidak, Anda mencari **penentuan kategori**, sehingga Anda akan menggunakan `logistic regression`. Anda akan mempelajari lebih lanjut tentang logistic regression nanti. Pikirkan beberapa pertanyaan yang dapat Anda ajukan dari data, dan metode mana yang lebih sesuai.

Di bagian ini, Anda akan bekerja dengan [dataset kecil tentang diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Bayangkan Anda ingin menguji pengobatan untuk pasien diabetes. Model Machine Learning dapat membantu Anda menentukan pasien mana yang akan merespons pengobatan dengan lebih baik, berdasarkan kombinasi variabel. Bahkan model regresi yang sangat dasar, ketika divisualisasikan, dapat menunjukkan informasi tentang variabel yang dapat membantu Anda mengatur uji klinis teoritis Anda.

Dengan demikian, mari kita mulai tugas ini!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>

<!--![Karya seni oleh \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Karya seni oleh @allison_horst-->


## 1. Memuat alat-alat kita

Untuk tugas ini, kita memerlukan paket-paket berikut:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) adalah [koleksi paket R](https://www.tidyverse.org/packages) yang dirancang untuk membuat ilmu data lebih cepat, mudah, dan menyenangkan!

-   `tidymodels`: Kerangka [tidymodels](https://www.tidymodels.org/) adalah [koleksi paket](https://www.tidymodels.org/packages/) untuk pemodelan dan pembelajaran mesin.

Anda dapat menginstalnya dengan:

`install.packages(c("tidyverse", "tidymodels"))`

Script di bawah ini memeriksa apakah Anda memiliki paket-paket yang diperlukan untuk menyelesaikan modul ini dan menginstalnya untuk Anda jika ada yang belum terpasang.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Sekarang, mari kita muat paket-paket luar biasa ini dan membuatnya tersedia di sesi R kita saat ini. (Ini hanya untuk ilustrasi, `pacman::p_load()` sudah melakukannya untuk Anda)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Dataset Diabetes

Dalam latihan ini, kita akan mempraktikkan keterampilan regresi dengan membuat prediksi menggunakan dataset diabetes. [Dataset diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) mencakup `442 sampel` data terkait diabetes, dengan 10 variabel fitur prediktor, yaitu `usia`, `jenis kelamin`, `indeks massa tubuh`, `tekanan darah rata-rata`, dan `enam pengukuran serum darah`, serta variabel hasil `y`: ukuran kuantitatif dari perkembangan penyakit satu tahun setelah baseline.

|Jumlah observasi|442|
|----------------|:---|
|Jumlah prediktor|10 kolom pertama adalah prediktor numerik|
|Hasil/Target|Kolom ke-11 adalah ukuran kuantitatif perkembangan penyakit satu tahun setelah baseline|
|Informasi Prediktor|- usia dalam tahun
||- jenis kelamin
||- bmi indeks massa tubuh
||- bp tekanan darah rata-rata
||- s1 tc, total kolesterol serum
||- s2 ldl, lipoprotein densitas rendah
||- s3 hdl, lipoprotein densitas tinggi
||- s4 tch, rasio total kolesterol / HDL
||- s5 ltg, kemungkinan log dari kadar trigliserida serum
||- s6 glu, kadar gula darah|

> 🎓 Ingat, ini adalah pembelajaran terawasi (supervised learning), dan kita memerlukan target bernama 'y'.

Sebelum Anda dapat memanipulasi data dengan R, Anda perlu mengimpor data ke dalam memori R, atau membangun koneksi ke data yang dapat digunakan R untuk mengakses data secara jarak jauh.

> Paket [readr](https://readr.tidyverse.org/), yang merupakan bagian dari Tidyverse, menyediakan cara yang cepat dan ramah untuk membaca data berbentuk persegi panjang ke dalam R.

Sekarang, mari kita muat dataset diabetes yang disediakan di URL sumber ini: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Selain itu, kita akan melakukan pemeriksaan awal pada data kita menggunakan `glimpse()` dan menampilkan 5 baris pertama menggunakan `slice()`.

Sebelum melangkah lebih jauh, mari kita juga mengenalkan sesuatu yang akan sering Anda temui dalam kode R 🥁🥁: operator pipe `%>%`

Operator pipe (`%>%`) melakukan operasi secara berurutan dengan meneruskan sebuah objek ke dalam fungsi atau ekspresi panggilan berikutnya. Anda dapat menganggap operator pipe seperti mengatakan "dan kemudian" dalam kode Anda.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` menunjukkan bahwa data ini memiliki 442 baris dan 11 kolom, dengan semua kolom bertipe data `double`.

<br>

> glimpse() dan slice() adalah fungsi dalam [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, bagian dari Tidyverse, adalah tata bahasa manipulasi data yang menyediakan serangkaian kata kerja konsisten untuk membantu Anda menyelesaikan tantangan manipulasi data yang paling umum.

<br>

Sekarang kita memiliki data, mari kita fokus pada satu fitur (`bmi`) untuk dijadikan target dalam latihan ini. Ini akan membutuhkan kita untuk memilih kolom yang diinginkan. Jadi, bagaimana cara melakukannya?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) memungkinkan kita untuk *memilih* (dan secara opsional mengganti nama) kolom dalam sebuah data frame.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Data Pelatihan dan Pengujian

Dalam pembelajaran terawasi, adalah praktik umum untuk *membagi* data menjadi dua subset; satu set (biasanya lebih besar) untuk melatih model, dan satu set "cadangan" yang lebih kecil untuk melihat bagaimana kinerja model.

Sekarang kita memiliki data yang siap, kita dapat melihat apakah mesin dapat membantu menentukan pembagian logis antara angka-angka dalam dataset ini. Kita dapat menggunakan paket [rsample](https://tidymodels.github.io/rsample/), yang merupakan bagian dari kerangka kerja Tidymodels, untuk membuat sebuah objek yang berisi informasi tentang *bagaimana* membagi data, dan kemudian dua fungsi rsample lainnya untuk mengekstrak set pelatihan dan pengujian yang telah dibuat:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Melatih model regresi linear dengan Tidymodels

Sekarang kita siap untuk melatih model kita!

Dalam Tidymodels, Anda menentukan model menggunakan `parsnip()` dengan menyebutkan tiga konsep:

-   **Tipe model** membedakan jenis model seperti regresi linear, regresi logistik, model pohon keputusan, dan sebagainya.

-   **Mode model** mencakup opsi umum seperti regresi dan klasifikasi; beberapa tipe model mendukung salah satu dari ini, sementara yang lain hanya memiliki satu mode.

-   **Mesin model** adalah alat komputasi yang akan digunakan untuk menyesuaikan model. Sering kali ini adalah paket R, seperti **`"lm"`** atau **`"ranger"`**

Informasi pemodelan ini ditangkap dalam spesifikasi model, jadi mari kita buat satu!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Setelah sebuah model *ditentukan*, model tersebut dapat `diestimasi` atau `dilatih` menggunakan fungsi [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), biasanya dengan menggunakan formula dan beberapa data.

`y ~ .` berarti kita akan memodelkan `y` sebagai kuantitas/target yang diprediksi, dijelaskan oleh semua prediktor/fitur yaitu `.` (dalam kasus ini, kita hanya memiliki satu prediktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Dari keluaran model, kita dapat melihat koefisien yang dipelajari selama pelatihan. Koefisien tersebut mewakili koefisien dari garis terbaik yang memberikan kesalahan keseluruhan terendah antara variabel aktual dan yang diprediksi.
<br>

## 5. Membuat prediksi pada set uji

Sekarang setelah kita melatih model, kita dapat menggunakannya untuk memprediksi progresi penyakit y pada dataset uji menggunakan [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Ini akan digunakan untuk menggambar garis di antara kelompok data.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 Kami baru saja melatih sebuah model dan menggunakannya untuk membuat prediksi!

Saat membuat prediksi, konvensi tidymodels adalah selalu menghasilkan tibble/data frame hasil dengan nama kolom yang sudah distandarisasi. Hal ini mempermudah penggabungan data asli dan prediksi dalam format yang dapat digunakan untuk operasi selanjutnya seperti membuat plot.

`dplyr::bind_cols()` secara efisien menggabungkan beberapa data frame berdasarkan kolom.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Plot hasil pemodelan

Sekarang saatnya melihat ini secara visual 📈. Kita akan membuat scatter plot dari semua nilai `y` dan `bmi` pada set pengujian, lalu menggunakan prediksi untuk menggambar garis di tempat yang paling sesuai, di antara pengelompokan data model.

R memiliki beberapa sistem untuk membuat grafik, tetapi `ggplot2` adalah salah satu yang paling elegan dan paling serbaguna. Ini memungkinkan Anda untuk menyusun grafik dengan **menggabungkan komponen independen**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Pikirkan sebentar tentang apa yang sedang terjadi di sini. Sebuah garis lurus melewati banyak titik kecil data, tetapi apa sebenarnya yang sedang dilakukan? Bisakah kamu melihat bagaimana garis ini seharusnya dapat digunakan untuk memprediksi di mana titik data baru yang belum terlihat seharusnya berada dalam hubungannya dengan sumbu y pada plot? Cobalah untuk menjelaskan dengan kata-kata kegunaan praktis dari model ini.

Selamat, kamu telah membuat model regresi linear pertama, membuat prediksi dengannya, dan menampilkannya dalam sebuah plot!



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa terjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
